In [1]:
# This notebook will demonstrate slicing

import numpy as np
import skimage as ski
print(np.__file__)
print(np.__version__)
print(ski.__version__)
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import matplotlib.pyplot as plt

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import tensorflow as tf
from skimage.transform import resize
from graphviz import Graph, render
from utils.netviz import NetViz
from utils.mnistutil import MNISTUitl
from utils.sliceutil import Slice
sx = 28
sy = 28

/Users/mislam/anaconda/lib/python3.6/site-packages/numpy/__init__.py
1.16.2
0.14.2


/Users/mislam/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
mn = MNISTUitl()
viz = NetViz()

In [3]:
X, Y, x, y = mn.getdata2(0,0,sx,sy)
X.shape

/Users/mislam/anaconda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(60000, 28, 28)

In [4]:
nm , xt, yt = mn.train2(X, Y, x,y,sx,sy,10,50)

x_train shape: (60000, 28, 28, 1)
(60000, 28, 28, 1) 60000 train samples (60000,)
10000 test samples
(60000,) (60000,)
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (Flatten)              (None, 784)               0         
_________________________________________________________________
H (Dense)                    (None, 49)                38465     
_________________________________________________________________
output (Dense)               (None, 10)                500       
Total params: 38,965
Trainable params: 38,965
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
60000/60000 [==============================] - 2s 29us/step - loss: 1.4473 - acc: 0.6293

In [5]:
X, Y, x, y = mn.getdata2(0,0,sx,sy)
model , xt1, yt1 = mn.train2(X, Y, x,y,sx,sy,10,50)

/Users/mislam/anaconda/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


x_train shape: (60000, 28, 28, 1)
(60000, 28, 28, 1) 60000 train samples (60000,)
10000 test samples
(60000,) (60000,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (Flatten)              (None, 784)               0         
_________________________________________________________________
H (Dense)                    (None, 49)                38465     
_________________________________________________________________
output (Dense)               (None, 10)                500       
Total params: 38,965
Trainable params: 38,965
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
60000/60000 [==============================] - 2s 29us/step - loss: 1.4528 - acc: 0.6637

In [6]:
#W1, W2, b1, b2 = slc.getweights(nm)

In [7]:
#W1, W2,b1,b2 = slc.dynamicmodify(nm,xt[0],sx,sy)

In [8]:
#W1.shape
nm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (Flatten)              (None, 784)               0         
_________________________________________________________________
H (Dense)                    (None, 49)                38465     
_________________________________________________________________
output (Dense)               (None, 10)                500       
Total params: 38,965
Trainable params: 38,965
Non-trainable params: 0
_________________________________________________________________


In [9]:
#W2.shape

In [143]:
zero  = []
slc = Slice()
W1, W2, b1, b2 = slc.getweights(nm)
for i in range(0,len(yt)):
    if yt[i] == 9:
        zero.append(xt[i])
print(len(zero),len(Y))

1009 60000


In [144]:
np.random.shuffle(zero)
zero = zero[0:500]
#slc.D1

In [145]:
for x in zero:
    W1, W2,b1,b2 = slc.dynamicmodify(nm,x,sx,sy)
    #W1, W2,b1,b2 = slc.modifyThroughInterSection(nm,x,sx,sy)

In [146]:
#dot, A = slc.showstatweights(nm,zero[0],[0],sx,sy,ss = .2)
#dot
slc.D1

array([[ 0.        ,  0.05154783,  0.        , ...,  0.01834673,
         0.        , -0.04666462],
       [ 0.        , -0.08099991,  0.        , ..., -0.04870778,
         0.        ,  0.04261176],
       [ 0.        ,  0.03100532,  0.        , ...,  0.02134512,
         0.        ,  0.01588812],
       ...,
       [ 0.        , -0.07010315,  0.        , ...,  0.02865963,
         0.        , -0.03992468],
       [ 0.        , -0.00989756,  0.        , ..., -0.07342958,
         0.        ,  0.03643977],
       [ 0.        ,  0.01046117,  0.        , ..., -0.06825088,
         0.        , -0.03724324]], dtype=float32)

In [147]:
#doto, A = viz.vispredictwights(nm, xt[0], yt[0] , sx, sy )
#doto

In [148]:
slc.D1[0]
nz = np.count_nonzero(slc.D1)
nz

26656

In [149]:
tot = 784*49
tot

38416

In [150]:
nz / tot

0.6938775510204082

In [151]:
np.count_nonzero(slc.D2)

392

In [152]:
np.count_nonzero(slc.W1)

38416

In [153]:
w,b = nm.layers[1].get_weights()

In [154]:
print(yt[0:1])
nm.predict(xt[0:1])

[7]


array([[4.5296906e-06, 7.1923512e-09, 1.7464004e-05, 1.0832150e-02,
        1.9497957e-08, 1.3206291e-05, 2.2960111e-11, 9.8897409e-01,
        8.6139780e-06, 1.4983017e-04]], dtype=float32)

In [155]:
model.layers[1].set_weights([slc.D1,slc.d1])
model.layers[2].set_weights([slc.D2,slc.d2])

In [156]:
from sklearn.metrics import accuracy_score
zeros = []
pred = []
tr = []
labs = [0,1,2,3,4,5,6,7,8,9]
acc = []
count = 0
for l in labs:
    pred = []
    tr = []
    for i in range(0,len(yt)):
        count += 1
        if yt[i] == l:
            p = model.predict(xt[i:i+1])
            m = p.argmax()
            pred.append(m)
            tr.append(l)
    score = accuracy_score(pred,tr)
    acc.append(score)
print(acc)
print(count)

[0.29897959183673467, 0.9806167400881057, 0.0, 0.9514851485148514, 0.9562118126272913, 0.9338565022421524, 0.9676409185803758, 0.9620622568093385, 0.9260780287474333, 0.9266600594648167]
100000
